In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Processing Day-Ahead and Real-Time Locational Marginal Price (LMP) Data 

In [2]:
# importing and merging the historical day ahead and real time locational marginal prices
lmp2015 = pd.read_csv('data/rt_da_monthly_lmps_2015_PEPCO.csv')
lmp2016 = pd.read_csv('data/rt_da_monthly_lmps_2016_PEPCO.csv')
lmp2017 = pd.read_csv('data/rt_da_monthly_lmps_2017_PEPCO.csv')
lmp2018 = pd.read_csv('data/rt_da_monthly_lmps_2018_PEPCO.csv')
lmp2019 = pd.read_csv('data/rt_da_monthly_lmps_2019_PEPCO.csv')
settled_lmps = lmp2015.append(lmp2016)
settled_lmps = settled_lmps.append(lmp2017)
settled_lmps = settled_lmps.append(lmp2018)
settled_lmps = settled_lmps.append(lmp2019)

In [3]:
# current month LMPs not yet published in the previously imported format,
# so individually imported from separate report
posted_rt_lmps = pd.read_csv('data/rt_hrl_lmps_11_2019_PEPCO.csv')
posted_da_lmps = pd.read_csv('data/da_hrl_lmps_11_2019_PEPCO.csv')

In [4]:
# removing unneccessary columns from settled data
settled_lmps.drop(columns=['voltage', 'equipment',
                           'type', 'zone'],
                  inplace=True)

In [5]:
# initializing a new dataframe
# with only important columns from current month's posted real-time data
posted_lmps = posted_rt_lmps.drop(columns=['voltage', 'equipment',
                                           'type', 'zone',
                                           'row_is_current', 'version_nbr'])

In [6]:
# populating the rest of the dataframe with day-ahead data
posted_lmps[['system_energy_price_da',
             'total_lmp_da',
             'congestion_price_da',
             'marginal_loss_price_da']] = posted_da_lmps[['system_energy_price_da',
                                                          'total_lmp_da',
                                                          'congestion_price_da',
                                                          'marginal_loss_price_da']]

In [7]:
# converting select columns to datetime
settled_lmps['datetime_beginning_utc'] = pd.to_datetime(settled_lmps['datetime_beginning_utc'])
settled_lmps['datetime_beginning_ept'] = pd.to_datetime(settled_lmps['datetime_beginning_ept'])

posted_lmps['datetime_beginning_utc'] = pd.to_datetime(posted_lmps['datetime_beginning_utc'])
posted_lmps['datetime_beginning_ept'] = pd.to_datetime(posted_lmps['datetime_beginning_ept'])

In [8]:
# ensuring no duplicates
settled_lmps.drop_duplicates(inplace=True)

posted_lmps.drop_duplicates(inplace=True)

In [9]:
# merging and sorting data and setting index to utc time
lmp_df = settled_lmps.append(posted_lmps)
lmp_df.sort_values(by='datetime_beginning_utc', inplace=True)
lmp_df.index = lmp_df['datetime_beginning_utc']
lmp_df.drop(columns='datetime_beginning_utc', inplace=True)

In [10]:
lmp_df.head(2)

,datetime_beginning_ept,pnode_id,pnode_name,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,system_energy_price_da,total_lmp_da,congestion_price_da,marginal_loss_price_da
datetime_beginning_utc,,,,,,,,,,,
2015-01-01 05:00:00,2015-01-01 00:00:00,51298,PEPCO,27.90,29.679524,0.518941,1.260583,28.11,32.515687,3.763290,0.642397
2015-01-01 06:00:00,2015-01-01 01:00:00,51298,PEPCO,27.89,29.381736,0.238513,1.253222,27.83,32.999114,4.533097,0.636017


In [11]:
lmp_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42840 entries, 2015-01-01 05:00:00 to 2019-11-21 04:00:00
Data columns (total 11 columns):
datetime_beginning_ept    42840 non-null datetime64[ns]
pnode_id                  42840 non-null int64
pnode_name                42840 non-null object
system_energy_price_rt    42840 non-null float64
total_lmp_rt              42840 non-null float64
congestion_price_rt       42840 non-null float64
marginal_loss_price_rt    42840 non-null float64
system_energy_price_da    42840 non-null float64
total_lmp_da              42840 non-null float64
congestion_price_da       42840 non-null float64
marginal_loss_price_da    42840 non-null float64
dtypes: datetime64[ns](1), float64(8), int64(1), object(1)
memory usage: 3.9+ MB


# Processing PEPCO, Mid-Atlantic, and Forecasted Mid-Atlantic Load (Instantaneous Consumption) Data 

In [12]:
# importing and merging the historical load (i.e. instantaneous consumption) data for pepco
pepco_load_2015 = pd.read_csv('data/hrl_load_metered_2015_PEPCO.csv')
pepco_load_2016 = pd.read_csv('data/hrl_load_metered_2016_PEPCO.csv')
pepco_load_2017 = pd.read_csv('data/hrl_load_metered_2017_PEPCO.csv')
pepco_load_2018 = pd.read_csv('data/hrl_load_metered_2018_PEPCO.csv')
pepco_load_2019 = pd.read_csv('data/hrl_load_metered_2019_PEPCO.csv')
pepco_load = pepco_load_2015.append(pepco_load_2016)
pepco_load = pepco_load.append(pepco_load_2017)
pepco_load = pepco_load.append(pepco_load_2018)
pepco_load = pepco_load.append(pepco_load_2019)

In [13]:
# importing and merging the historical load (i.e. instantaneous consumption) data for mid-atlantic
midatl_load_2015 = pd.read_csv('data/hrl_load_metered_2015_MIDATL.csv')
midatl_load_2016 = pd.read_csv('data/hrl_load_metered_2016_MIDATL.csv')
midatl_load_2017 = pd.read_csv('data/hrl_load_metered_2017_MIDATL.csv')
midatl_load_2018 = pd.read_csv('data/hrl_load_metered_2018_MIDATL.csv')
midatl_load_2019 = pd.read_csv('data/hrl_load_metered_2019_MIDATL.csv')
midatl_load = midatl_load_2015.append(midatl_load_2016)
midatl_load = midatl_load.append(midatl_load_2017)
midatl_load = midatl_load.append(midatl_load_2018)
midatl_load = midatl_load.append(midatl_load_2019)

In [14]:
# importing and merging the historical load (i.e. instantaneous consumption) forecasts for mid-atlantic
# historical pepco forecasts not available
midatl_frcstd_load_2015 = pd.read_csv('data/load_frcstd_hist_2015_MIDATL.csv')
midatl_frcstd_load_2016 = pd.read_csv('data/load_frcstd_hist_2016_MIDATL.csv')
midatl_frcstd_load_2017 = pd.read_csv('data/load_frcstd_hist_2017_MIDATL.csv')
midatl_frcstd_load_2018 = pd.read_csv('data/load_frcstd_hist_2018_MIDATL.csv')
midatl_frcstd_load_2019 = pd.read_csv('data/load_frcstd_hist_2019_MIDATL.csv')
midatl_frcstd_load = midatl_frcstd_load_2015.append(midatl_frcstd_load_2016)
midatl_frcstd_load = midatl_frcstd_load.append(midatl_frcstd_load_2017)
midatl_frcstd_load = midatl_frcstd_load.append(midatl_frcstd_load_2018)
midatl_frcstd_load = midatl_frcstd_load.append(midatl_frcstd_load_2019)

In [15]:
# making explicit the different loads
pepco_load.rename({'mw': 'pepco_mw'}, axis=1, inplace=True)

midatl_load.rename({'mw': 'midatl_mw'}, axis=1, inplace=True)

midatl_frcstd_load.rename({'forecast_load_mw': 'midatl_forecast_load_mw'}, axis=1, inplace=True)

In [16]:
# removing unneccessary columns
pepco_load.drop(columns=['datetime_beginning_ept', 'nerc_region',
                         'mkt_region', 'zone',
                         'load_area', 'is_verified'],
                inplace=True)

midatl_load.drop(columns=['datetime_beginning_ept', 'nerc_region',
                          'mkt_region', 'zone',
                          'load_area', 'is_verified'],
                 inplace=True)

midatl_frcstd_load.drop(columns=['forecast_hour_beginning_ept',
                                 'forecast_area'], inplace=True)

In [17]:
# converting select columns to datetime
pepco_load['datetime_beginning_utc'] = pd.to_datetime(pepco_load['datetime_beginning_utc'])

midatl_load['datetime_beginning_utc'] = pd.to_datetime(midatl_load['datetime_beginning_utc'])

midatl_frcstd_load['evaluated_at_utc'] = pd.to_datetime(midatl_frcstd_load['evaluated_at_utc'])
midatl_frcstd_load['evaluated_at_ept'] = pd.to_datetime(midatl_frcstd_load['evaluated_at_ept'])
midatl_frcstd_load['forecast_hour_beginning_utc'] = pd.to_datetime(midatl_frcstd_load['forecast_hour_beginning_utc'])

In [18]:
# ensuring no duplicates
pepco_load.drop_duplicates(inplace=True)

midatl_load.drop_duplicates(inplace=True)

midatl_frcstd_load.drop_duplicates(inplace=True)

In [19]:
# sorting data and setting index to utc time
pepco_load.sort_values(by='datetime_beginning_utc', inplace=True)
pepco_load.index = pepco_load['datetime_beginning_utc']
pepco_load.drop(columns='datetime_beginning_utc', inplace=True)

midatl_load.sort_values(by='datetime_beginning_utc', inplace=True)
midatl_load.index = midatl_load['datetime_beginning_utc']

midatl_frcstd_load.sort_values(by=['forecast_hour_beginning_utc', 'evaluated_at_utc'], inplace=True)
midatl_frcstd_load.reset_index(drop=True, inplace=True)

In [20]:
# mid-atlantic load is provided as a separate line for each sub-region for each hour
# so it must be aggregated by hour to have a single value for each hour
midatl_load = midatl_load.resample('1H').sum()

In [21]:
# merging the pepco and mid-atlantic load data into a common load dataframe
load_df = pepco_load.merge(midatl_load, how='left', left_index=True, right_index=True)

In [22]:
# only retaining the oldest and latest forecast for simplicity
temp_midatl_frcstd_load = midatl_frcstd_load.copy()
# identifying the oldest and latest forecast indices
hrly_midatl_frcstd_load = temp_midatl_frcstd_load.groupby(by='forecast_hour_beginning_utc').idxmin()
hrly_midatl_frcstd_load_last = temp_midatl_frcstd_load.groupby(by='forecast_hour_beginning_utc').idxmax()
# resetting/establishing dataframes with the oldest and latest forecasts
midatl_frcstd_load = temp_midatl_frcstd_load.iloc[hrly_midatl_frcstd_load.iloc[:,0], :].reset_index(drop=True)
midatl_frcstd_load_last = temp_midatl_frcstd_load.iloc[hrly_midatl_frcstd_load_last.iloc[:,0], :].reset_index(drop=True)

In [23]:
# combining the two forecast dataframes and setting the index to utc time
midatl_frcstd_load['midatl_forecast_load_mw_final'] = midatl_frcstd_load_last.loc[:, 'midatl_forecast_load_mw']
midatl_frcstd_load.index = midatl_frcstd_load.loc[:, 'forecast_hour_beginning_utc']
midatl_frcstd_load.drop(columns=['forecast_hour_beginning_utc'], inplace=True)

In [24]:
# merging the historical load forecasts into the load dataframe
load_df = load_df.merge(midatl_frcstd_load, how='left', left_index=True, right_index=True)

In [25]:
load_df.tail(2)

,pepco_mw,midatl_mw,evaluated_at_utc,evaluated_at_ept,midatl_forecast_load_mw,midatl_forecast_load_mw_final
datetime_beginning_utc,,,,,,
2019-11-21 03:00:00,2650.363,29184.714,2019-11-19 10:45:00,2019-11-19 05:45:00,29481,29500
2019-11-21 04:00:00,2487.477,27440.844,2019-11-19 10:45:00,2019-11-19 05:45:00,27716,27711


In [26]:
# generating a common dataframe with lmp and load data
data_df = lmp_df.merge(load_df, how='left', left_index=True, right_index=True)
data_df.index.name = 'datetime_beginning_utc'

In [27]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42840 entries, 2015-01-01 05:00:00 to 2019-11-21 04:00:00
Data columns (total 17 columns):
datetime_beginning_ept           42840 non-null datetime64[ns]
pnode_id                         42840 non-null int64
pnode_name                       42840 non-null object
system_energy_price_rt           42840 non-null float64
total_lmp_rt                     42840 non-null float64
congestion_price_rt              42840 non-null float64
marginal_loss_price_rt           42840 non-null float64
system_energy_price_da           42840 non-null float64
total_lmp_da                     42840 non-null float64
congestion_price_da              42840 non-null float64
marginal_loss_price_da           42840 non-null float64
pepco_mw                         42840 non-null float64
midatl_mw                        42840 non-null float64
evaluated_at_utc                 42840 non-null datetime64[ns]
evaluated_at_ept                 42840 non-null datetime64[ns

# Processing Regional Interface Flow (Electricity Transmitted In/Out of Region) Data 

In [29]:
# importing and merging interface flow (i.e electricity transmitted in/out of regions) data
interface_flows_2015 = pd.read_csv('data/da_interface_flows_and_limits_2015_ALL.csv')
interface_flows_2016 = pd.read_csv('data/da_interface_flows_and_limits_2016_ALL.csv')
interface_flows_2017 = pd.read_csv('data/da_interface_flows_and_limits_2017_ALL.csv')
interface_flows_2018 = pd.read_csv('data/da_interface_flows_and_limits_2018_ALL.csv')
interface_flows_2019 = pd.read_csv('data/da_interface_flows_and_limits_2019_ALL.csv')
interface_flows_stack = interface_flows_2015.append(interface_flows_2016)
interface_flows_stack = interface_flows_stack.append(interface_flows_2017)
interface_flows_stack = interface_flows_stack.append(interface_flows_2018)
interface_flows_stack = interface_flows_stack.append(interface_flows_2019)

In [30]:
# removing unnecessary columns
interface_flows_stack.drop(columns=['datetime_beginning_ept'], inplace=True)

In [31]:
interface_flows_stack.head()

,datetime_beginning_utc,interface_limit_name,flow_mw,limit_mw
0,1/1/2015 5:00:00 AM,APSOUTH,4463,4900
1,1/1/2015 5:00:00 AM,BCPEP,5281,5600
2,1/1/2015 5:00:00 AM,BED-BLA,1590,1600
3,1/1/2015 5:00:00 AM,CENTRAL,1540,3000
4,1/1/2015 5:00:00 AM,EAST,3944,5300


In [32]:
# data is provided as a single column of flows with a separate row for each region and hour
# initializing a dataframe with one row per hour in the data
# that will be appended with one column per regions' flow and flow limit
interface_flows = interface_flows_stack[['datetime_beginning_utc']].copy()
interface_flows.drop_duplicates(inplace=True)

In [33]:
# performing the unstacking by grouping the data by region
# and overwritting the copied dataframe
grouped = interface_flows_stack.groupby(by='interface_limit_name')
for name, group in grouped:
    group = group.copy()
    # naming the columns based on region
    group.rename(columns={'flow_mw': name.lower().replace('-', '_')+'_flow_mw',
                          'limit_mw': name.lower().replace('-', '_')+'_limit_mw'},
                 inplace=True)
    # dropping name column that is now represented in the flow column names
    group.drop(columns=['interface_limit_name'], inplace=True)
    # appending the region columns to the final dataframe
    interface_flows = interface_flows.merge(group,
                                            how='left',
                                            left_on='datetime_beginning_utc',
                                            right_on='datetime_beginning_utc')

In [34]:
interface_flows.head()

,datetime_beginning_utc,apsouth_flow_mw,apsouth_limit_mw,bcpep_flow_mw,bcpep_limit_mw,bed_bla_flow_mw,bed_bla_limit_mw,central_flow_mw,central_limit_mw,east_flow_mw,east_limit_mw,west_flow_mw,west_limit_mw
0,1/1/2015 5:00:00 AM,4463,4900,5281,5600,1590.0,1600.0,1540,3000,3944,5300,4385,5200
1,1/1/2015 6:00:00 AM,4399,4900,5164,5600,1592.0,1600.0,1419,3000,3621,5300,4128,5200
2,1/1/2015 7:00:00 AM,4377,4900,5035,5600,1597.0,1600.0,1581,3000,3818,5300,4152,5200
3,1/1/2015 8:00:00 AM,4431,4900,5164,5600,1599.0,1600.0,1516,3000,3812,5300,4205,5200
4,1/1/2015 9:00:00 AM,4338,4900,5127,5600,1598.0,1600.0,1549,3000,3734,5300,4211,5200


In [35]:
# converting select columns to datetime
interface_flows['datetime_beginning_utc'] = pd.to_datetime(interface_flows['datetime_beginning_utc'])

In [36]:
# ensuring no duplicates again
interface_flows.drop_duplicates(inplace=True)

In [37]:
# sorting data and setting index to utc time
interface_flows.sort_values(by='datetime_beginning_utc', inplace=True)
interface_flows.index = interface_flows['datetime_beginning_utc']
interface_flows.drop(columns='datetime_beginning_utc', inplace=True)

In [38]:
interface_flows.describe()

,apsouth_flow_mw,apsouth_limit_mw,bcpep_flow_mw,bcpep_limit_mw,bed_bla_flow_mw,bed_bla_limit_mw,central_flow_mw,central_limit_mw,east_flow_mw,east_limit_mw,west_flow_mw,west_limit_mw
count,43128.000000,43128.000000,43128.000000,43128.000000,41760.000000,41760.000000,43128.000000,43128.000000,43128.000000,43128.000000,43128.000000,43128.000000
mean,2295.892413,3543.274346,3618.109094,5484.862201,954.976509,1840.806298,-616.100237,3167.992951,2898.959725,5991.569282,1522.926614,4756.079577
std,979.412525,748.365616,1139.612433,825.455692,371.802535,6054.665558,1392.003529,883.571095,1684.549295,1214.009497,1651.943750,899.913377
min,-382.000000,1100.000000,165.000000,2300.000000,-602.000000,0.000000,-3596.000000,200.000000,-1314.000000,1200.000000,-3056.000000,1600.000000
25%,1593.000000,3100.000000,2779.000000,4900.000000,687.000000,1300.000000,-1624.000000,2600.000000,1677.000000,5200.000000,313.000000,4100.000000
50%,2205.000000,3500.000000,3609.000000,5500.000000,921.000000,1400.000000,-883.000000,3200.000000,2704.000000,6000.000000,1298.000000,4800.000000
75%,2889.000000,3900.000000,4425.000000,6000.000000,1200.000000,1600.000000,110.000000,3800.000000,3940.000000,6800.000000,2573.000000,5400.000000
max,5909.000000,9999.000000,7200.000000,9999.000000,2195.000000,99999.000000,4843.000000,5500.000000,9837.000000,10200.000000,6500.000000,7100.000000


In [39]:
# removing erroneous values
interface_flows['bcpep_limit_mw'] = interface_flows['bcpep_limit_mw'].apply(lambda x: np.nan if x >= 9999 else x)
interface_flows['bed_bla_limit_mw'] = interface_flows['bed_bla_limit_mw'].apply(lambda x: np.nan if x >= 9999 else x)
interface_flows['apsouth_limit_mw'] = interface_flows['apsouth_limit_mw'].apply(lambda x: np.nan if x >= 9999 else x)

In [40]:
# merging interface flows into common dataframe
data_df = data_df.merge(interface_flows, how='left', left_index=True, right_index=True)

In [41]:
data_df.info()
# some missing interface values, which will eventually be filled via interpolation

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42840 entries, 2015-01-01 05:00:00 to 2019-11-21 04:00:00
Data columns (total 29 columns):
datetime_beginning_ept           42840 non-null datetime64[ns]
pnode_id                         42840 non-null int64
pnode_name                       42840 non-null object
system_energy_price_rt           42840 non-null float64
total_lmp_rt                     42840 non-null float64
congestion_price_rt              42840 non-null float64
marginal_loss_price_rt           42840 non-null float64
system_energy_price_da           42840 non-null float64
total_lmp_da                     42840 non-null float64
congestion_price_da              42840 non-null float64
marginal_loss_price_da           42840 non-null float64
pepco_mw                         42840 non-null float64
midatl_mw                        42840 non-null float64
evaluated_at_utc                 42840 non-null datetime64[ns]
evaluated_at_ept                 42840 non-null datetime64[ns

# Processing Forecasted Generator Outage Data 

In [42]:
# importing and merging forecasted generation outage data
# note this data is provided on a daily interval, 
# not hourly like most of the other data
gen_outages_2015 = pd.read_csv('data/gen_outages_by_type_2015.csv')
gen_outages_2016 = pd.read_csv('data/gen_outages_by_type_2016.csv')
gen_outages_2017 = pd.read_csv('data/gen_outages_by_type_2017.csv')
gen_outages_2018 = pd.read_csv('data/gen_outages_by_type_2018.csv')
gen_outages_2019 = pd.read_csv('data/gen_outages_by_type_2019.csv')
gen_outages = gen_outages_2015.append(gen_outages_2016)
gen_outages = gen_outages.append(gen_outages_2017)
gen_outages = gen_outages.append(gen_outages_2018)
gen_outages = gen_outages.append(gen_outages_2019)

In [43]:
# only retaining data for the region that contains pepco
gen_outages = gen_outages.loc[gen_outages['region']=='Mid Atlantic - Dominion']

In [44]:
# removing unnecessary columns and resetting index
gen_outages.drop(columns=['region'], inplace=True)
gen_outages.reset_index(drop=True, inplace=True)

In [45]:
# converting select columns to datetime
gen_outages['forecast_execution_date_ept'] = pd.to_datetime(gen_outages['forecast_execution_date_ept'])
gen_outages['forecast_date'] = pd.to_datetime(gen_outages['forecast_date'])

In [46]:
# ensuring no duplicates
gen_outages.drop_duplicates(inplace=True)

In [47]:
# will only keep the previous day's forecast as this is what would be used in a deployed model
gen_outages['day_delta'] = gen_outages['forecast_date'] - gen_outages['forecast_execution_date_ept']

kept_delta = pd.Timedelta('1 days 00:00:00')

gen_outages = gen_outages.loc[gen_outages['day_delta'] == kept_delta]

gen_outages.drop(columns=['day_delta'], inplace=True)

In [48]:
# renaming select columns
gen_outages.rename({'forecast_execution_date_ept': 'outage_frcst_execution_date',
                    'forecast_date': 'outage_forecast_date'}, axis=1, inplace=True)

In [49]:
# sorting data and setting index to the date
gen_outages.sort_values(by='outage_forecast_date', inplace=True)
gen_outages.index = gen_outages['outage_forecast_date']
gen_outages.drop(columns='outage_forecast_date', inplace=True)

In [50]:
# resampling to populate the daily value for each hour of a given day
gen_outages = gen_outages.resample('H').pad()

In [51]:
gen_outages.head(2)

,outage_frcst_execution_date,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw
outage_forecast_date,,,,,
2015-05-27 00:00:00,2015-05-26,15001,9827,3336,1838
2015-05-27 01:00:00,2015-05-26,15001,9827,3336,1838


In [52]:
# merging outages into common dataframe
data_df = data_df.merge(gen_outages, how='left', left_index=True, right_index=True)

In [53]:
data_df.info()
# outage data only extends back to mid 2015,
# so those time periods will be excluded from the final data that is modeled

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42840 entries, 2015-01-01 05:00:00 to 2019-11-21 04:00:00
Data columns (total 34 columns):
datetime_beginning_ept           42840 non-null datetime64[ns]
pnode_id                         42840 non-null int64
pnode_name                       42840 non-null object
system_energy_price_rt           42840 non-null float64
total_lmp_rt                     42840 non-null float64
congestion_price_rt              42840 non-null float64
marginal_loss_price_rt           42840 non-null float64
system_energy_price_da           42840 non-null float64
total_lmp_da                     42840 non-null float64
congestion_price_da              42840 non-null float64
marginal_loss_price_da           42840 non-null float64
pepco_mw                         42840 non-null float64
midatl_mw                        42840 non-null float64
evaluated_at_utc                 42840 non-null datetime64[ns]
evaluated_at_ept                 42840 non-null datetime64[ns

# Processing Weather Data 

In [54]:
# importing and merging weather data
weather_2015_2016 = pd.read_csv('data/KDCA_weather_data_2015-2016.csv')
weather_2017_2018 = pd.read_csv('data/KDCA_weather_data_2017-2018.csv')
weather_2019 = pd.read_csv('data/KDCA_weather_data_2019-20191121.csv')
weather = weather_2015_2016.append(weather_2017_2018)
weather = weather.append(weather_2019)

In [55]:
# dropping unnecessary outages
weather.drop(columns=['latitude', 'longitude',
                      'timezone', 'type',
                      'summary', 'icon'],
             inplace=True)

In [56]:
# converting select columns to datetime
weather['time'] = pd.to_datetime(weather['time'])

In [57]:
# ensuring no duplicates
weather.drop_duplicates(inplace=True)

In [58]:
# sorting and setting index to time
weather.sort_values(by='time', inplace=True)
weather.index = weather['time']
weather.drop(columns='time', inplace=True)

In [59]:
# the API request and data saving process filled missing precipitation types with 'none',
# but some of those correspond to times with a non-zero precipitation intensity
# inspection of the data showed they all appeared to represent times of rain,
# so they are populated accordingly
weather.loc[(weather['precipType'] == 'none') &
            (weather['precipIntensity'] > 0), ['precipType']] = 'rain'

In [60]:
weather.head(2)

,precipIntensity,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility
time,,,,,,,,,,,,,,
2015-01-01 05:00:00,0.0,0.0,none,23.90,23.90,15.19,0.69,1024.4,0.03,0.29,262.0,0.00,0.0,7.884
2015-01-01 06:00:00,0.0,0.0,none,23.37,23.37,15.27,0.71,1024.4,0.05,0.72,270.0,0.08,0.0,8.251


In [61]:
# merging weather data into the common dataframe
data_df = data_df.merge(weather, how='left', left_index=True, right_index=True)

In [62]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42840 entries, 2015-01-01 05:00:00 to 2019-11-21 04:00:00
Data columns (total 48 columns):
datetime_beginning_ept           42840 non-null datetime64[ns]
pnode_id                         42840 non-null int64
pnode_name                       42840 non-null object
system_energy_price_rt           42840 non-null float64
total_lmp_rt                     42840 non-null float64
congestion_price_rt              42840 non-null float64
marginal_loss_price_rt           42840 non-null float64
system_energy_price_da           42840 non-null float64
total_lmp_da                     42840 non-null float64
congestion_price_da              42840 non-null float64
marginal_loss_price_da           42840 non-null float64
pepco_mw                         42840 non-null float64
midatl_mw                        42840 non-null float64
evaluated_at_utc                 42840 non-null datetime64[ns]
evaluated_at_ept                 42840 non-null datetime64[ns

In [63]:
# some missing values, 
# close inspection of data showed there are never more than a handful of consecutive missing values,
# so interpolation is used to populate those values in the weather and interface flow data
data_df.interpolate(method='time', axis=0, inplace=True)

In [64]:
# categorical can't be interpolated, if no measured precipitiation then 'none' populated
data_df.loc[(data_df['precipType'].isna()) &
            (data_df['precipIntensity'] == 0), ['precipType']] = 'none'

In [65]:
# inspected nearby values to confirm the missing types when precipitation was measured
# appear to be for rain
data_df.loc[(data_df['precipType'].isna()) &
            (data_df['precipIntensity'] > 0), ['precipType']] = 'rain'

# Processing Sunlight Data 

In [66]:
# importing sunrise and sunset data
sun_time_data = pd.read_csv('data/sunset_sunrise_2015-2020.csv')

In [67]:
# dropping unnecessary columns
sun_time_data.drop(columns=['solar_noon_lst', 'sunlight_duration_mins'], inplace=True)

In [68]:
# generating strings with both the date and sunrise/set times
sun_time_data['sunrise_datetime_lst'] = sun_time_data['date']+' '+sun_time_data['sunrise_time_lst']
sun_time_data['sunset_datetime_lst'] = sun_time_data['date']+' '+sun_time_data['sunset_time_lst']

In [69]:
# converting the dates and sunrise/set times to datetimes
sun_time_data['date'] = pd.to_datetime(sun_time_data['date'])
sun_time_data['sunrise_datetime_lst'] = pd.to_datetime(sun_time_data['sunrise_datetime_lst'])
sun_time_data['sunset_datetime_lst'] = pd.to_datetime(sun_time_data['sunset_datetime_lst'])

# converting times to utc to allow for merging and handling of daylight savings
sun_time_data['utc_offset'] = pd.Timedelta('0 days 05:00:00')
sun_time_data['sunrise_datetime_utc'] = sun_time_data['sunrise_datetime_lst'] + sun_time_data['utc_offset']
sun_time_data['sunset_datetime_utc'] = sun_time_data['sunset_datetime_lst'] + sun_time_data['utc_offset']

In [70]:
# only retaining important columns and setting the index to the date
sun_time_data = sun_time_data[['date', 'sunrise_datetime_utc', 'sunset_datetime_utc']]

sun_time_data.index = sun_time_data['date']
sun_time_data.drop(columns=['date'], inplace=True)

In [71]:
# resample to have a row for each hour based on the day's values
sun_time_data = sun_time_data.resample('H').pad()

In [72]:
# merging sun time data into the common dataframe
data_df = data_df.merge(sun_time_data, how='left', left_index=True, right_index=True)

In [73]:
# creating timedelta constants for 1 and zero hours
ONE_HOUR = pd.Timedelta('0 days 01:00:00')
ZERO_HOUR = pd.Timedelta('0 days 00:00:00')

In [74]:
len(data_df.index)

42840

In [75]:
data_df.drop_duplicates(inplace=True)
len(data_df.index)

42840

In [76]:
# generating a column ('sunniness') that will contain the number of seconds of sunlight in a given hour
# first step is to create detlas between each hour and the day's sunset and sunrise time 
# sunrise requires calculating delta from end of hour to rise time to represent period with light
data_df['sunrise_delta'] = data_df.index + pd.Timedelta('0 days 01:00:00') - data_df['sunrise_datetime_utc']
# sunset requires hour start to be subtracted from set time to represent period with light
data_df['sunset_delta'] = data_df['sunset_datetime_utc'] - data_df.index

# if the sunrise delta is negative then populate it with 0
# (i.e. before sunrise so no sunlight)
# else populate it with the lesser of the delta and 1
# (i.e. 1 for all hours after sunrise except the one in which sunrise occurs)
data_df['sunrise_sunniness'] = data_df['sunrise_delta'].apply(lambda x:
                                                              ZERO_HOUR if x < ZERO_HOUR
                                                              else min(x, ONE_HOUR))
# if the sunset delta is negative then populate it with 0
# (i.e. after sunset so no sunlight)
# else populate it with the lesser of the delta and 1
# (i.e. 1 for all hours before sunset except the one in which sunset occurs)
data_df['sunset_sunniness'] = data_df['sunset_delta'].apply(lambda x:
                                                            ZERO_HOUR if x < ZERO_HOUR
                                                            else min(x, ONE_HOUR))

# now the information in the sunrise and sunset sunniness columns are combined with the minimum value retained
# if either has a 0 for a given hour then it is entirely before sunrise or after sunset,
# and the zero is retained by the minimum function.
# if one is 1 and the other is a fraction or 1 then that lower value is retained
# (i.e. sunrise/set hour and full daylight hours)
data_df['sunniness'] = pd.DataFrame([data_df['sunrise_sunniness'], data_df['sunset_sunniness']]).min()
data_df['sunniness'] = data_df['sunniness'].dt.total_seconds()

In [77]:
# dropping unnecessary intermediate calculation columns
data_df.drop(columns=['sunrise_delta', 'sunset_delta', 'sunrise_sunniness',
                      'sunset_sunniness'], inplace=True)

# Exporting Final Data to CSV

In [78]:
# dropping hours with missing outage data from early 2015
data_df.dropna(axis=0, how='any', inplace=True)

In [79]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39341 entries, 2015-05-27 00:00:00 to 2019-11-21 04:00:00
Data columns (total 51 columns):
datetime_beginning_ept           39341 non-null datetime64[ns]
pnode_id                         39341 non-null int64
pnode_name                       39341 non-null object
system_energy_price_rt           39341 non-null float64
total_lmp_rt                     39341 non-null float64
congestion_price_rt              39341 non-null float64
marginal_loss_price_rt           39341 non-null float64
system_energy_price_da           39341 non-null float64
total_lmp_da                     39341 non-null float64
congestion_price_da              39341 non-null float64
marginal_loss_price_da           39341 non-null float64
pepco_mw                         39341 non-null float64
midatl_mw                        39341 non-null float64
evaluated_at_utc                 39341 non-null datetime64[ns]
evaluated_at_ept                 39341 non-null datetime64[ns

In [80]:
# dropping descriptive data that won't be considered for modeling
# will retain this data in a separate dataframe and csv just in case, 
# but further work will only be performed on data_clean
data_clean = data_df.drop(columns=['pnode_id', 'pnode_name',
                                   'evaluated_at_utc', 'evaluated_at_ept',
                                   'sunrise_datetime_utc', 'sunset_datetime_utc',
                                   'outage_frcst_execution_date', 'windBearing'])

In [81]:
# since part of 2015 data was incomplete the full year will be excluded for modeling purposes.
# the last 8 days of 2015 are retained for potential use as lag variables.
data_df = data_df.loc[data_df['datetime_beginning_ept'] >= pd.Timestamp(2015, 12, 24)].copy()
data_clean = data_clean.loc[data_clean['datetime_beginning_ept'] >= pd.Timestamp(2015, 12, 24)].copy()

In [82]:
data_df.head(2)

,datetime_beginning_ept,pnode_id,pnode_name,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,system_energy_price_da,total_lmp_da,congestion_price_da,marginal_loss_price_da,pepco_mw,midatl_mw,evaluated_at_utc,evaluated_at_ept,midatl_forecast_load_mw,midatl_forecast_load_mw_final,apsouth_flow_mw,apsouth_limit_mw,bcpep_flow_mw,bcpep_limit_mw,bed_bla_flow_mw,bed_bla_limit_mw,central_flow_mw,central_limit_mw,east_flow_mw,east_limit_mw,west_flow_mw,west_limit_mw,outage_frcst_execution_date,total_outages_mw,planned_outages_mw,maintenance_outages_mw,forced_outages_mw,precipIntensity,precipProbability,precipType,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,sunrise_datetime_utc,sunset_datetime_utc,sunniness
datetime_beginning_utc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-24 05:00:00,2015-12-24 00:00:00,51298,PEPCO,11.53,13.691565,1.967474,0.194091,14.11,16.192954,2.015654,0.067300,2193.423,23323.247,2015-12-23 10:45:00,2015-12-23 05:45:00,26248,23622,1273.0,2700.0,2880.0,5300.0,557.0,1300.0,-2828.0,2500.0,434.0,4500.0,-734.0,4200.0,2015-12-23,8519.0,1525.0,5415.0,1579.0,0.0,0.0,none,65.73,66.39,63.71,0.93,1011.3,3.32,9.52,205.0,0.96,0.0,8.365,2015-12-24 12:24:20,2015-12-24 21:50:59,0.0
2015-12-24 06:00:00,2015-12-24 01:00:00,51298,PEPCO,2.11,2.480393,0.329983,0.040411,10.63,12.922192,2.231668,0.060524,2101.570,22048.989,2015-12-23 10:45:00,2015-12-23 05:45:00,25061,22405,1489.0,2700.0,2820.0,5300.0,606.0,1300.0,-2848.0,2500.0,442.0,4500.0,-747.0,4200.0,2015-12-23,8519.0,1525.0,5415.0,1579.0,0.0,0.0,none,66.02,66.62,63.47,0.91,1010.6,4.12,10.70,206.0,0.80,0.0,8.571,2015-12-24 12:24:20,2015-12-24 21:50:59,0.0


In [127]:
data_df.to_csv('data/processed/data.csv')

In [128]:
data_clean.to_csv('data/processed/data_clean.csv')